In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams
import os
import re
%matplotlib inline

from rapidfuzz import fuzz, process, utils
import jellyfish
import snowflake.connector

import time
# from time import time

import pandasql

pd.options.display.max_colwidth = 10000
pd.set_option('display.max_rows', 100000)


In [2]:
cnx = snowflake.connector.connect(user='rkothari', password='Fall19@123', account='FF80747.east-us-2.azure', warehouse='DASCI_WH_S', database='FDC_STAGE', schema='DASCI', role='DASCI_DEVELOPER')


In [3]:
q = """SELECT * FROM FDC_PROD_LIVE.FDC02.ENTITY_INVOICE I
WHERE  INVOICE_UPDATE_DATETIME BETWEEN '2021-08-10 13:00:00.000' AND '2021-08-10 14:00:00.000' 
"""

In [4]:
ENTITY = pd.read_sql(q,cnx)

In [5]:
len(ENTITY)

453455

In [6]:
ENTITY.to_csv("/home/rkothari/Downloads/ENTITY.csv")

In [7]:
q2 = """  SELECT distinct INVOICE_UPC_CASE as PRODUCT_UPC_CASE ,GTIN,
PRODUCT_DESCRIPTION from FDC_STAGE.DASCI.RK_PROD_CAT_MAPPED_TEMP"""

In [8]:
RK_PROD_CAT_MAPPED = pd.read_sql(q2,cnx)

In [9]:
RK_PROD_CAT_MAPPED.to_csv("/home/rkothari/Downloads/RK_PROD_CAT_MAPPED.csv")

In [10]:
data = pandasql.sqldf("SELECT DISTINCT DISTRIBUTOR_FDC_ID,DISTRIBUTOR_FTS_ID,INVOICE_UPDATE_DATETIME,INVOICE_UPC_CASE,INVOICE_UPC_PACK,INVOICE_PACKAGE_DESCRIPTION , GTIN, PRODUCT_DESCRIPTION FROM \
                      ENTITY,RK_PROD_CAT_MAPPED WHERE INVOICE_UPC_CASE = PRODUCT_UPC_CASE OR INVOICE_UPC_PACK = PRODUCT_UPC_CASE")

In [11]:
data.to_csv("/home/rkothari/Downloads/master_data.csv")

In [29]:
data2 = pandasql.sqldf("SELECT DISTINCT DISTRIBUTOR_FDC_ID,DISTRIBUTOR_FTS_ID,INVOICE_UPDATE_DATETIME,INVOICE_UPC_CASE,INVOICE_UPC_PACK,INVOICE_PACKAGE_DESCRIPTION , GTIN, PRODUCT_DESCRIPTION FROM \
                      ENTITY,RK_PROD_CAT_MAPPED WHERE INVOICE_UPC_CASE = PRODUCT_UPC_CASE \
                      union \
                    SELECT DISTINCT DISTRIBUTOR_FDC_ID,DISTRIBUTOR_FTS_ID,INVOICE_UPDATE_DATETIME,INVOICE_UPC_CASE,INVOICE_UPC_PACK,INVOICE_PACKAGE_DESCRIPTION , GTIN, PRODUCT_DESCRIPTION FROM \
                      ENTITY,RK_PROD_CAT_MAPPED WHERE  \
                       INVOICE_UPC_PACK = PRODUCT_UPC_CASE")

In [30]:
len(data2)

573573

In [12]:
matched_description = data[data['INVOICE_PACKAGE_DESCRIPTION'] == data['PRODUCT_DESCRIPTION']]

In [13]:
len(matched_description)

1048

In [14]:
# % Match
len(matched_description)/len(data)

0.001827143188399733

In [15]:
len(data)

573573

In [16]:
filtered_data = data[~data['INVOICE_PACKAGE_DESCRIPTION'].isin(matched_description.INVOICE_PACKAGE_DESCRIPTION)]

In [17]:
len(filtered_data)

571724

In [18]:
filtered_data['new_invoice_description'] = ''

In [19]:
filtered_data.reset_index(inplace=True)

In [20]:
def calculate_soundex_ratio(x,y):
    Xsoundex = jellyfish.soundex(x)
    Ysoundex = jellyfish.soundex(y)
    return calculate_fuzz_ratio(Xsoundex,Ysoundex)

In [21]:
def calculate_fuzz_ratio(x,y):
    return fuzz.partial_ratio(x,y)

In [22]:
def mapping_logic(invoice_description,product_description, fuzzy_threshold, soundex_threshold):
    flag = "NEED TO INSPECT"
#Fuzzy call
    fuzzy_ratio = calculate_fuzz_ratio(invoice_description ,product_description)

    if fuzzy_ratio >= fuzzy_threshold:
        
        return product_description
    else:
#Soundex Call
        soundex_fuzz_ratio = calculate_soundex_ratio(invoice_description ,product_description)
        if soundex_fuzz_ratio >= soundex_threshold:
            return product_description 
        else:
            return flag


In [23]:
def left(string, length):
    return string[:length]

def right(string, length):
    return string[-length:]

In [24]:
GTIN_LENGTH =  [14,13,12,11,10]
def mapping_function(data,i, match_upc, fuzzy_threshold, soundex_threshold):
    flag = "NEED TO INSPECT"
    invoice_description     = data['INVOICE_PACKAGE_DESCRIPTION'][i]
    
    product_description     = data['PRODUCT_DESCRIPTION'][i]
#    invoice_product_is_id   = data['PRODUCT_IS_ID'][0]
#    product_master_key      = data['PRODUCT_MASTER_CATALOG_KEY'][0]
    GTIN                    = data['GTIN'][i]
    
    
    
    for N in GTIN_LENGTH:
    
        if len(GTIN) == 14:                                   #Length 14 
                if invoice_description != product_description:    # if description doesn't not match
                    return mapping_logic(invoice_description,product_description, fuzzy_threshold, soundex_threshold)        #Call mapping logic
        elif right(GTIN,N) == match_upc and invoice_description != product_description:
            
            return mapping_logic(invoice_description,product_description, fuzzy_threshold, soundex_threshold) 
        elif left(GTIN,N) == match_upc and invoice_description != product_description:
            
            return mapping_logic(invoice_description,product_description, fuzzy_threshold, soundex_threshold) 
        else:
            return flag



            

            
        


In [25]:

for i in range(0,len(filtered_data)):

    filtered_data['new_invoice_description'][i] = mapping_function(filtered_data,i,filtered_data['INVOICE_UPC_CASE'][i],50,70)


for i in range(0,len(filtered_data)):
    for n in GTIN_LENGTH:
        if  filtered_data['new_invoice_description'][i] == "NEED TO INSPECT":
            
            filtered_data['new_invoice_description'][i] = mapping_function(filtered_data,i,filtered_data['INVOICE_UPC_PACK'][i],50,70)



In [27]:
need_to_inspect = filtered_data[filtered_data['new_invoice_description'] == 'NEED TO INSPECT']


In [28]:
len(need_to_inspect)

17073